<a href="https://colab.research.google.com/github/meghbhalerao/nrlpq/blob/main/NLRPQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch.nn as nn
import torch
import torchvision
import numpy as np
from torch.utils.data import Subset
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
from torchvision import models
import torch.nn.functional as F
import copy

Mounted at /content/gdrive


In [2]:
num_classes = 10

In [3]:
nesting_start=3

class BlurPoolConv2d(torch.nn.Module):
    def __init__(self, conv):
        super().__init__()
        default_filter = torch.tensor([[[[1, 2, 1], [2, 4, 2], [1, 2, 1]]]]) / 16.0
        filt = default_filter.repeat(conv.in_channels, 1, 1, 1)
        self.conv = conv
        self.register_buffer('blur_filter', filt)

    def forward(self, x):
        blurred = F.conv2d(x, self.blur_filter, stride=1, padding=(1, 1),
                           groups=self.conv.in_channels, bias=None)
        return self.conv.forward(blurred)

class Model():
    def __init__(self, gpu, nesting, single_head, fixed_feature, use_blurpool):
        super().__init__()
        self.gpu = gpu
        self.nesting = nesting
        self.sh = single_head
        self.ff = fixed_feature
        self.use_blurpool = use_blurpool


    def load_model(self, model, model_weights_disk, modify_keys = True):
        if os.path.isfile(model_weights_disk):
            print("=> loading checkpoint '{}'".format(model_weights_disk))
            if self.gpu is None:
                checkpoint = torch.load(model_weights_disk)
            else:
                # Map model to be loaded to specified single gpu.
                loc = 'cuda:{}'.format(self.gpu)
                checkpoint = torch.load(model_weights_disk, map_location=loc)
            if modify_keys:
              checkpoint = self.change_str_dict(checkpoint)
            


            try:
              model.load_state_dict(checkpoint)
            except:
              proxy_layer = nn.Linear(2048, num_classes)
              print("randomly init last fc layer")
              checkpoint["fc.bias"] = proxy_layer.bias
              checkpoint["fc.weight"] = proxy_layer.weight
              print(checkpoint.keys())
              model.load_state_dict(checkpoint)

            print("=> loaded checkpoint '{}' "
                  .format(model_weights_disk))
        else:
            print("=> no model found at '{}'".format(model_weights_disk))

        return model


    def change_str_dict(self, x):
      x_ = {}
      for key, val in x.items():
        key_new = str(key).replace("module.","")
        x_[key_new] = copy.copy(val)
      return x_

    def initModel(self):
        print("Model init: nesting=%d, sh=%d, ff=%d" %(self.nesting, self.sh, self.ff))
        model = models.resnet50(pretrained=True)
        nesting_list = [2**i for i in range(nesting_start, 12)] if self.nesting else None

        # Nesting/Fixed Feature Modification code block
        if self.nesting:
            ff= "Single Head" if self.sh else "Multi Head"
            print("Using Nesting of type - {}".format(ff))
            print("Nesting Starts from {}".format(2**nesting_start))
            if self.sh:
                model.fc =  SingleHeadNestedLinear(nesting_list, num_classes=num_classes)
            else:
                model.fc =  MultiHeadNestedLinear(nesting_list, num_classes=num_classes)
        elif self.ff != 2048:
            print(f"Using Fixed Features = {self.ff}")
            model.fc =  FixedFeatureLayer(self.ff, num_classes)

        def apply_blurpool(mod: torch.nn.Module):
            for (name, child) in mod.named_children():
                if isinstance(child, torch.nn.Conv2d) and (np.max(child.stride) > 1 and child.in_channels >= 16):
                    setattr(mod, name, BlurPoolConv2d(child))
                else: apply_blurpool(child)
        if self.use_blurpool: apply_blurpool(model)

        model = model.to(memory_format=torch.channels_last)
        model = model.to(self.gpu)

        return model


import torch
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional

class SingleHeadNestedLinear(nn.Linear):
	def __init__(self, nesting_list: List, num_classes=num_classes, **kwargs):
		super(SingleHeadNestedLinear, self).__init__(nesting_list[-1], num_classes, **kwargs)
		self.nesting_list=nesting_list
		self.num_classes=num_classes # Number of classes for classification

	def forward(self, x):
		nesting_logits = ()
		for i, num_feat in enumerate(self.nesting_list):
			if not (self.bias is None):
				logit = torch.matmul(x[:, :num_feat], (self.weight[:, :num_feat]).t()) + self.bias
			else:
				logit = torch.matmul(x[:, :num_feat], (self.weight[:, :num_feat]).t())
			nesting_logits+= (logit,)
		return nesting_logits

class MultiHeadNestedLinear(nn.Module):
	def __init__(self, nesting_list: List, num_classes=num_classes, **kwargs):
		super(MultiHeadNestedLinear, self).__init__()
		self.nesting_list=nesting_list
		self.num_classes=num_classes # Number of classes for classification
		for i, num_feat in enumerate(self.nesting_list):
			setattr(self, f"nesting_classifier_{i}", nn.Linear(num_feat, self.num_classes, **kwargs))		

	def forward(self, x):
		nesting_logits = ()
		for i, num_feat in enumerate(self.nesting_list):
			nesting_logits +=  (getattr(self, f"nesting_classifier_{i}")(x[:, :num_feat]),)
		return nesting_logits

		
class FixedFeatureLayer(nn.Linear):
    # This layer just takes the first "K" Features for the classification. 
    # Creating a separate layer and customized fwd pass helps to not change the base codes at all.
    def __init__(self, in_features, out_features, **kwargs):
        super(FixedFeatureLayer, self).__init__(in_features, out_features, **kwargs)

    def forward(self, x):
        if not (self.bias is None):
            out = torch.matmul(x[:, :self.in_features], self.weight.t()) + self.bias
        else:
            out = torch.matmul(x[:, :self.in_features], self.weight.t())
        return out

class NestedCELoss(nn.Module):
	def __init__(self, **kwargs):
		super(NestedCELoss, self).__init__()
		self.criterion = nn.CrossEntropyLoss(**kwargs)
	def forward(self, output, target):
		loss=0
		for o in output:
			loss+= self.criterion(o, target)

		return loss

In [5]:

def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, device):
    train_losses = []
    valid_losses = []

    for epoch in range(0, epochs):
        # training
        print(f'EPOCH:{epoch}')
        print('Computing Train Loss..')
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
      
        print(train_loss)
        train_losses.append(train_loss)

        # validation
        with torch.no_grad():
            print('Computing Validation Loss..')
            model, valid_loss = validate(valid_loader, model, criterion, device)
            print(valid_loss)
            valid_losses.append(valid_loss)
        print('--------------------------------')
  
    return model, train_losses, valid_losses

def train(train_loader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0
    for X, y in train_loader:
        optimizer.zero_grad()
        X = X.to(device)
        y = y.to(device)
        pred = model(X) 
        loss = criterion(pred, y) 
        running_loss += loss.item() * X.size(0)
        loss.backward()
        optimizer.step()
      #  print(loss.cpu().data.item())
        
    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss

def validate(valid_loader, model, criterion, device):
    model.eval()
    running_loss = 0
    correct_count, all_count = 0, 0
    
    for X, y in valid_loader:
        X = X.to(device)
        y = y.to(device)
        pred = model(X) 
        
        loss = criterion(pred, y) 
        running_loss += loss.item() * X.size(0)

    epoch_loss = running_loss / len(valid_loader.dataset)
    return model, epoch_loss

def get_accuracy(model,data_loader,rep_no, device):
  correct_count, all_count = 0, 0
  for images,labels in data_loader:
    images,labels = images.to(device), labels.to(device)
    images = images.to(device)
    true_label = labels.to(device)
    pred = model(images) 
    pred = pred[rep_no]
    pred_label = torch.argmax(pred, dim=1)
    correct_count += torch.eq(true_label, pred_label).sum().item()
    all_count +=len(true_label)
  return correct_count/all_count
    

In [7]:
mdl_wts = os.path.join("/content/gdrive/MyDrive/nrlpq/Imagenet1k_R50_sh1_mh0_ns3_ff2048/final_weights.pt")
model_wts_path = os.path.join(mdl_wts)
nesting = 1
single_head = 1
fixed_feature = 2048
model = Model(0, nesting, single_head, fixed_feature, use_blurpool=1)


model_init = model.initModel()
model = model.load_model(model_init, model_wts_path)
print("Loaded pretrained model: " + str(model_wts_path))


batch_size = 128
print("batch size is", batch_size)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset_all = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
n_alltrain = len(trainset_all)
print("len of all train data is", n_alltrain)

n_train = int(n_alltrain * 0.8)
n_val = n_alltrain - n_train
print("len of train val split is ", n_train, n_val, "respectively")

val_idxs = np.random.choice(n_alltrain, size = n_val ,replace=False)

trainset = Subset(trainset_all, list(set(range(len(trainset_all))) -  set(val_idxs)))
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

valset = Subset(trainset_all, val_idxs)
val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)


print("length of train, val and test DataLoader is ", len(train_loader), len(val_loader), len(test_loader))

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = NestedCELoss()
epochs = 1
device = torch.device('cuda')
model,_,_ = training_loop(model, criterion, optimizer, train_loader, val_loader, epochs, device)
res = get_accuracy(model, test_loader,3, device)
print(res)

Model init: nesting=1, sh=1, ff=2048
Using Nesting of type - Single Head
Nesting Starts from 8
=> no model found at '/content/gdrive/MyDrive/nrlpq/Imagenet1k_R50_sh1_mh0_ns3_ff2048/final_weights.pt'
Loaded pretrained model: /content/gdrive/MyDrive/nrlpq/Imagenet1k_R50_sh1_mh0_ns3_ff2048/final_weights.pt
batch size is 128
Files already downloaded and verified
len of all train data is 50000
len of train val split is  40000 10000 respectively
Files already downloaded and verified
length of train, val and test DataLoader is  313 79 79
EPOCH:0
Computing Train Loss..
9.938902421569825
Computing Validation Loss..
7.365488230895996
--------------------------------


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [28]:
layer_dict = {'layer1':[["conv1", "bn1"], ["conv2", "bn2"], ["conv3", "bn3", 'relu'] ]}

In [29]:
# Fuse the model in place rather manually.
model.to(device)
# Make a copy of the model for layer fusion
fused_model = copy.deepcopy(model)

model.eval()
# The model has to be switched to evaluation mode before any layer fusion.
# Otherwise the quantization will not work correctly.
fused_model.eval()

fused_model = torch.quantization.fuse_modules(fused_model, [["conv1", "bn1", "relu"]], inplace=True)
for module_name, module in fused_model.named_children():
    if "layer" in module_name:
        for basic_block_name, basic_block in module.named_children():
            torch.quantization.fuse_modules(basic_block, layer_dict[module_name], inplace=True)
            for sub_block_name, sub_block in basic_block.named_children():
                if sub_block_name == "downsample":
                    torch.quantization.fuse_modules(sub_block, [["0", "1"]], inplace=True)

KeyError: ignored

In [30]:
fused_model

ResNet(
  (conv1): ConvReLU2d(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(inplace=True)
  )
  (bn1): Identity()
  (relu): Identity()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
      (conv3): ConvReLU2d(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU(inplace=True)
      )
      (bn3): Identity()
      (relu): Identity()
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): Identity()
      )
    )
    (1): Bottleneck(
      (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddi